In [27]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from pickle import load

In [1]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [2]:
# load text
raw_text = load_doc('rhyme.txt')
print(raw_text)

Sing a song of sixpence,
A pocket full of rye.
Four and twenty blackbirds,
Baked in a pie.

When the pie was opened
The birds began to sing;
Wasn't that a dainty dish,
To set before the king.

The king was in his counting house,
Counting out his money;
The queen was in the parlour,
Eating bread and honey.

The maid was in the garden,
Hanging out the clothes,
When down came a blackbird
And pecked off her nose.


In [3]:
# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)

In [4]:
# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
    # select sequence of tokens
    seq = raw_text[i-length:i+1]
    # store
    sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 399


In [6]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [7]:
# save sequences to file
out_filename = 'char_sequencestutorial.txt'
save_doc(sequences, out_filename)

In [8]:
# load
in_filename = 'char_sequencestutorial.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

In [9]:
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))

In [10]:
sequences = list()
for line in lines:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    sequences.append(encoded_seq)

In [11]:
# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 38


In [16]:
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

In [17]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

In [18]:
# define model
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 75)                34200     
_________________________________________________________________
dense_1 (Dense)              (None, 38)                2888      
Total params: 37,088
Trainable params: 37,088
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
 - 1s - loss: 3.6085 - acc: 0.1429
Epoch 2/100
 - 0s - loss: 3.5066 - acc: 0.2030
Epoch 3/100
 - 0s - loss: 3.1910 - acc: 0.1905
Epoch 4/100
 - 0s - loss: 3.0478 - acc: 0.1905
Epoch 5/100
 - 0s - loss: 3.0226 - acc: 0.1905
Epoch 6/100
 - 0s - loss: 2.9825 - acc: 0.1905
Epoch 7/100
 - 0s - loss: 2.9722 - acc: 0.1905
Epoch 8/100
 - 0s - loss: 2.9477 - acc: 0.1905
Epoch 9/100
 - 0s - loss: 2.9238 - acc: 0.1905
Epoch 10/100
 - 0s - loss: 2.8951 - acc: 0.1905
Epoch 11/100
 - 0s - loss: 2.8813 - acc: 0.2080
Epoch 12/100
 - 0s - loss: 2.8484 - acc: 0.2206
Epoch 13/100
 - 0s - loss: 2.8083 - acc: 0.2105
Epoch 14/100
 - 0s - loss: 2.7729 - acc: 0.2180
Epoch 15/100
 - 0s - loss: 2.7536 - acc: 0.2155
Epoch 16/100
 - 0s - loss: 2.7232 - acc: 0.2306
Epoch 17/100
 - 0s - loss: 2.6610 - acc: 0.2331
Epoch 18/100
 - 0s - loss: 2.6383 - acc: 0.2657
Epoch 19/100
 - 0s - loss: 2.5870 - acc: 0.2882
Epoch 20/100
 - 0s - loss: 2.5544 - acc: 0.3008
Epoch 21/100
 - 0s - loss: 2.5050 - acc: 0.3158
E

In [20]:
# save the model to file
model.save('modeltutorial.h5')

In [21]:
# save the mapping
dump(mapping, open('mappingtutorial.pkl', 'wb'))

In [24]:
# load the model
model = load_model('modeltutorial.h5')

In [28]:
# load the mapping
mapping = load(open('mappingtutorial.pkl', 'rb'))

In [30]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

In [34]:
# test start of rhyme
print(generate_seq(model, mapping, 10, 'Sing a son', 20))
# test mid-line
print(generate_seq(model, mapping, 10, 'king was i', 20))
# test not in original
print(generate_seq(model, mapping, 10, 'four and pi', 20))

Sing a song of sixpence, A poc
king was in his counting house
four and pinnty hhe moue .he pa
